# Libraries

In [ ]:
import numpy as np
import pandas as pd
import random
from sklearn.metrics import accuracy_score
from collections import Counter
from keras.models import load_model
from utils import label_binary_to_dec
from resources.data_utils import split_dataset, DataGenerator
from resources.model_utils import evaluate_model
from resources.utils import label_str_to_dec,prediction_standardized,aggregate_generator_labels

# Load data

In [ ]:
dataset = 'dataset_name'
path = "/home/datasets/"+dataset+'/'
path_plaintext = path
path_final = path + "preprocessed_80/"
path_out = '/home/majority_vote/'
if not os.path.exists(path_out):
    os.makedirs(path_out)
    
char_list = [str(i) for i in range(1, num_classes+1)]
char2int = {char: i for i, char in enumerate(char_list)}

filename        = path_plaintext + 'labels_spot_binary.csv'
num_classes     = 15
num_replicates  = 1000
dataset_size    = num_classes * num_replicates
image_dimension = (80, 80)

In [ ]:
# Labels
labels = []
with open(filename, "r") as csvFile:
    for row in csvFile:
        labels.append(row[:-1])
labels = np.asarray(labels)
main_labels = label_str_to_dec(labels[0:dataset_size], char2int)

# Patterns
image_prefix = "FImg_ID_"
image_suffix = ".jpg"
images_str = ["{}{}{}{}".format(path_final, image_prefix, img_idx, image_suffix) for img_idx in range(1, dataset_size + 1)]
main_dataset = pd.DataFrame({"img_path": images_str, "label": main_labels})

# Split data sets 
generation_params = {"dim": image_dimension,\
                     "nb_classes": num_classes,\
                     "column_img": "img_path",\
                     "column_label": "label",}
df_train, df_valid, df_test = split_dataset(data_frame=main_dataset,
                                            rank=1,
                                            column_label="label",
                                            random_state=25)
train_generator = DataGenerator(data_frame=df_train, batch_size=16, shuffle=True, **generation_params)
valid_generator = DataGenerator(data_frame=df_valid, batch_size=100, shuffle=False, **generation_params)
test_generator = DataGenerator(data_frame=df_test, batch_size=100, shuffle=False, **generation_params)

# Load trained model

In [ ]:
path_model = '/home/trained_models/model.h5'
trained_model = load_model(path_model)

In [ ]:
# Get predictions
y_test_pred = label_binary_to_dec(prediction_standardized(evaluate_model(trained_model, test_generator, 6)))
# Get true labels
y_test_true = label_binary_to_dec(aggregate_generator_labels(data_generator=test_generator)) 
# Compute accuracy of using 1 pattern
acc_test = accuracy_score(y_test_true, y_test_pred)

# Multiple patterns for transmission

In [ ]:
n = 3 # number of sampels to use for decoding
majority_vote= []
for i in range(len(y_test_true)):
    y = y_test_true[i]
    idx = np.where(y_test_true == y)
    idx = list(idx[0])
    idx.remove(i)
    idx = random.sample(idx, n-1)
    idx = idx+[i]
    n_pred = y_test_pred[idx]    
    mv = Counter(n_pred)
    mv = mv.most_common(1)[0][0]
    majority_vote.append(mv)

In [ ]:
# Compute majority vote accuracy
vote_accuracy = accuracy_score(y_test_true,majority_vote)
print('Trianing size: '+str(training_dataset_size))
print('Original accuracy: '+str(acc_test))
print('Majority vote: '+str(vote_accuracy))